In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# get list of geo entities that need semantic universals
import json

with open("/content/drive/MyDrive/geonames_train.json", "r") as json_data:
    geonames_json = json.loads(json_data.read())
    json_data.close()

print(geonames_json[0])

{'ID': 0, 'term': 'Pic de Font Blanca', 'type': 'peak'}


In [2]:
# get all geo entities
entities_list = [i['type'] for i in geonames_json]
# remove all duplicate entities
entities = list(set(entities_list))
len(entities)

660

In [3]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.7 MB/s eta 0:00:00


In [4]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [22]:
# get id of entity

import requests

def get_entity_id(entity):
  API_ENDPOINT = "https://www.wikidata.org/w/api.php"

  #entity = "intermittent stream"

  # handle entities that are mislabeled:
  if entity == "tablemounts (or guyots)" or entity == "tablemount (or guyot)":
    query = "guyot"
  elif entity == "blowout(s)":
    query = "blowout"
  elif entity == "temple(s)":
    query = "temple"
  elif entity == "nursery(-ies)":
    query = "nursery"
  elif entity == "irrigation ditch":
    query = "ditch"
  elif entity == "ravine(s)":
    query = "ravine"
  elif entity == "cuesta(s)":
    query = "cuesta"
  elif entity == "oil pumping station":
    query = "pumping station"
  elif entity == "fossilized forest":
    query = "petrified forest"
  elif entity == "seat of government of a political entity":
    query = "seat of government"
  elif entity == "mud flat(s)":
    query = "mudflat"
  elif entity == "cave(s)":
    query = "cave"
  elif entity == "seat of a second-order administrative division":
    query = "administrative territorial entity"
  elif entity == "abandoned railroad stop":
    query = "abandoned railway"
  elif entity == "historical fifth-order administrative division":
    query = "historical administrative division"
  elif entity == "cone(s)":
    query = "cone"
    return "Q1635267" # "cone" is a special case whereby the first id result does not refer to the geographical definition
  elif entity == "escarpment (or scarp)":
    query = "escarpment"
  elif entity == "abandoned mission":
    query = "mission station"
  elif entity == "chrome mine(s)":
    query = "chromium mine"
  elif entity == "depression(s)":
    query = "depression" # "depression" is also a special entity that has more than one meaning and we want to target the geographical meaning
    return "Q190429"
  elif entity == "camp(s)":
    query = "camp"
    return "Q1326028" # "camp" is a special case whereby the first id result does not refer to the geographical definition
  elif entity == "former sugar mill":
    query = "sugarcane mill"
  elif entity == "ridge(s)":
    query = "ridge"
  elif entity == "burial cave(s)":
    query = "burial cave"
  elif entity == "gorge(s)":
    query = "gorge"
  elif entity == "historical capital of a political entity" or entity == "capital of a political entity":
    query = "capital city"
  elif entity == "field(s)":
    query = "field"
    return "Q188869" # "field" is a special case whereby the first id result does not refer to the geographical definition
  elif entity == "semi-independent political entity":
    query = "special administrative region" # "semi-independent political entity" is a special case whereby the first id result does not refer to the geographical definition
    return "Q779415"
  elif entity == "facility center":
    query = "facility"
    return "Q13226383" # for entity disambiguation
  elif entity == "seachannels":
    query = "abyssal channel"
  elif entity == "estate(s)":
    query = "estate"
    return "Q12292478" # for entity disambiguation
  elif entity == "mine(s)":
    query = "mine"
    return "Q820477"
  elif entity == "blowhole(s)":
    query = "blowhole"
    return "Q389545" # for entity disambiguation
  elif entity == "historical third-order administrative division":
    query = "third-order administrative division"
  elif entity == "bog(s)":
    query = "bog"
  elif entity == "lock(s)":
    query = "lock"
    return "Q105731" # for entity disambiguation
  elif entity == "populated locality":
    query = "locality"
    return "Q3257686" # for entity disambiguation
  elif entity == "plain(s)":
    query = "plain"
  elif entity == "seat of a fifth-order administrative division":
    query = "fifth-level administrative division"
  elif entity == "section of reef":
    query = "reef"
  elif entity == "archaeological/prehistoric site":
    query = "archaeological site"
  elif entity == "freely associated state":
    query = "free state"
  elif entity == "spring(s)":
    query = "spring"
    return "Q124714" # for entity disambiguation
  elif entity == "promontory(-ies)":
    query = "promontory"
  elif entity == "leased area":
    query = "lease"
  elif entity == "lake channel(s)":
    query = "lake channel"
  elif entity == "ranch(es)":
    query = "ranch"
  elif entity == "reservoir(s)":
    query = "reservoir"
    return "Q131681" # for entity disambiguation
  elif entity == "railroad stop":
    query = "railway station"
    return "Q55488" # for entity disambiguation
  elif entity == "slope(s)":
    query = "slope"
    return "Q944588"
  elif entity == "cleft(s)":
    query = "cleft"
  elif entity == "section of waterfall(s)":
    query = "waterfall"
    return "Q34038" # for entity disambiguation
  elif entity == "bight(s)":
    query = "bight"
    return "Q17018380" # for entity disambiguation
  elif entity == "oil pipeline terminal":
    query = "oil depot"
  elif entity == "moor(s)":
    query = "raised bog"
  elif entity == "toll gate/barrier":
    query = "tollgate"
  elif entity == "quarry(-ies)":
    query = "quarry"
    return "Q188040" # for entity disambiguation
  elif entity == "tidal flat(s)":
    query = "tidal flat"
  elif entity == "ruin(s)":
    query = "ruin"
  elif entity == "stream mouth(s)":
    query = "river mouth"
  elif entity == "agricultural reserve":
    query = "agricultural land"
  elif entity == "iron mine(s)":
    query = "iron mine"
  elif entity == "wharf(-ves)":
    query = "wharf"
    return "Q828909" # for entity disambiguation
  elif entity == "atoll(s)":
    query = "atoll"
  elif entity == "united states government establishment":
    query = "organization"
    return "Q43229" # for entity disambiguation
  elif entity == "tomb(s)":
    query = "tomb"
  elif entity == "hot spring(s)":
    query = "hot spring"
  elif entity == "sabkha(s)":
    query = "sabkha"
  elif entity == "historical second-order administrative division":
    query = "second-level administrative division"
  elif entity == "mound(s)":
    query = "mound"
    return "Q1584134" # for entity disambiguation
  elif entity == "section of harbor":
    query = "harbor"
  elif entity == "orchard(s)":
    query = "orchard"
  elif entity == "intermittent ponds":
    query = "intermittent pond"
  elif entity == "marsh(es)":
    query = "marsh"
  elif entity == "gold mine(s)":
    query = "gold mine"
  elif entity == "temp work office":
    query = "office"
  elif entity == "independent political entity" or entity == "section of independent political entity":
    query = "political entity"
  elif entity == "seat of a first-order administrative division":
    query = "first-level administrative division"
  elif entity == "ditch mouth(s)":
    query = "ditch mouth"
  elif entity == "coal mine(s)":
    query = "coal mine"
    return "Q959309" # for entity disambiguation
  elif entity == "dune(s)":
    query = "dune"
  elif entity == "sulphur spring(s)":
    query = "sulphur spring"
    return "Q12054316" # for entity disambiguation
  elif entity == "harbor(s)":
    query = "harbor"
  elif entity == "gas-oil separator plant":
    query = "Gas oil separation plant"
  elif entity == "mesa(s)":
    query = "mesa"
  elif entity == "waterfall(s)":
    query = "waterfall"
  elif entity == "butte(s)":
    query = "butte"
  elif entity == "cultivated area":
    return "Q18450613"
  elif entity == "gasfield":
    query = "natural gas field"
  elif entity == "historical first-order administrative division":
    query = "first-level administrative division"
  elif entity == "reef(s)":
    query = "reef"
  elif entity == "forest(s)":
    query = "forest"
  elif entity == "salt mine(s)":
    query = "salt mine"
  elif entity == "phosphate works":
    query = "phosphate industry"
  elif entity == "cliff(s)":
    query = "cliff"
  elif entity == "dependent political entity":
    query = "dependent territory"
  elif entity == "crater(s)":
    query = "crater"
  elif entity == "lake bed(s)":
    query = "former lake"
  elif entity == "spur(s)" or entity == "spur":
    query = "spur"
    return "Q1400565" # for entity disambiguation
  elif entity == "oasis(-es)":
    query = "oasis"
  elif entity == "administrative facility":
    query = "government building"
  elif entity == "wadies":
    query = "wadi"
  elif entity == "corral(s)":
    query = "pen"
    return "Q858177" # for entity disambiguation
  elif entity == "bank(s)":
    query = "ocean bank"
  elif entity == "railroad siding":
    query = "track"
    return "Q160342" # for entity disambiguation
  elif entity == "religious populated place":
    query = "holy place"
  elif entity == "tidal creek(s)":
    query = "tidal creek"
  elif entity == "palm tree reserve":
    query = "protected area"
  elif entity == "fan(s)":
    query = "alluvial fan"
  elif entity == "navigation canal(s)":
    query = "navigation channel"
  elif entity == "weir(s)":
    query = "weir"
  elif entity == "concession area":
    query = "concession"
    return "Q1192403" # for entity disambiguation
  elif entity == "arrugado":
    query = "washboarding"
  elif entity == "hammock(s)":
    query = "hammock"
    return "Q3126543" # for entity disambiguation
  elif entity == "underground irrigation canal(s)":
    query = "underground irrigation canal"
  elif entity == "mill(s)":
    query = "mill"
  elif entity == "abandoned police post":
    query = "cultural heritage"
  elif entity == "historical fourth-order administrative division":
    query = "fourth-level administrative division"
  elif entity == "bush(es)":
    query = "bush"
  elif entity == "tree(s)":
    query = "tree"
  elif entity == "abandoned airfield":
    query = "aerodrome"
  elif entity == "waterhole(s)":
    query = "waterhole"
  elif entity == "historical political entity":
    query = "former political party"
  elif entity == "interdune trough(s)":
    query = "interdune trough"
  elif entity == "coral reef(s)":
    query = "coral reef"
  elif entity == "aquaculture facility":
    query = "underwater site"
  elif entity == "agricultural facility":
    query = "agricultural building"
  elif entity == "distributary(-ies)":
    query = "distributary"
    return "Q591942" # for entity disambiguation
  elif entity == "cove(s)":
    query = "cove"
  elif entity == "glacier(s)":
    query = "glacier"
  elif entity == "pool(s)":
    query = "pool"
  elif entity == "dock(s)":
    query = "dock"
  elif entity == "seat of a fourth-order administrative division":
    query = "fourth-level administrative division"
  elif entity == "copper mine(s)":
    query = "copper mine"
  elif entity == "inspection station":
    query = "vehicle inspection"
  elif entity == "abandoned railroad station":
    query = "former railway station"
  elif entity == "seat of a third-order administrative division":
    query = "third-level administrative division"
  elif entity == "building(s)":
    query = "building"
  elif entity == "shoal(s)":
    query = "shoal"
  else:
    query = entity

  params = {
      'action': 'wbsearchentities',
      'format': 'json',
      'language': 'en',
      'search': query
  }

  r = requests.get(API_ENDPOINT, params = params)

  print("*** ENTITY: ", entity)
  print("*** DICT: ",r.json()['search'][0])
  entity_dict = r.json()['search'][0]
  return entity_dict['id']

In [6]:
# get semantic information for entity
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

def get_entity_semantic_information(entity_id):
  sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

  # entity id for 'mountain' retrieved from wikidata api
  #entity_id = 'wd:Q8502'

  # From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
  sparql.setQuery("""
  SELECT ?item ?itemLabel ?subclass ?subclassLabel ?instance ?instanceLabel ?part ?partLabel ?category ?categoryLabel ?properties ?propertiesLabel ?desc ?descLabel {
    VALUES (?item) { (wd:"""+entity_id+""") }
    VALUES ?missing { "n/a" }
    OPTIONAL { ?item wdt:P31 ?instance . }
    OPTIONAL { ?item wdt:P279 ?subclass . }
    OPTIONAL { ?item wdt:P361 ?part . }
    OPTIONAL { ?item wdt:P373 ?category . }
    OPTIONAL { ?item wdt:P1963 ?properties . }
    OPTIONAL { ?item schema:description ?desc .
            FILTER(LANG(?desc) = "en") .}
    BIND(if(bound(?subclass)  , ?subclassLabel , ?missing)  as ?subclassLabel)
    BIND(if(bound(?instance)  , ?instanceLabel , ?missing)  as ?instanceLabel)
    BIND(if(bound(?part)  , ?partLabel , ?missing)  as ?partLabel)
    BIND(if(bound(?category)  , ?categoryLabel , ?missing)  as ?categoryLabel)
    BIND(if(bound(?properties)  , ?propertiesLabel , ?missing)  as ?propertiesLabel)
    BIND(if(bound(?desc)  , ?descLabel , ?missing)  as ?descLabel)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  }
  """)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  results_df = pd.json_normalize(results['results']['bindings'])
  #results_df[['itemLabel.value', 'subclassLabel.value', 'instanceLabel.value', 'partLabel.value', 'categoryLabel.value', 'propertiesLabel.value', 'descLabel.value']].head()
  return results_df

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# transform description field into noun groups to get semantic universal nouns for a given entity
def get_description_noun_groups(results_df):
  word_list = []
  for txt in results_df["descLabel.value"].values:
    word_list.append([word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(txt)) if pos[0] == 'N'])
  results_df["descLabel.noungroups"] = word_list
  results_df["descLabel.noungroups"] = [','.join(item) for item in results_df["descLabel.noungroups"]]
  results_df = results_df.apply(lambda x: x.astype(str).str.lower())
  #results_df.head(5)
  return results_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
# generate semantic primes for entity
def generate_semantic_primes(results_df):
  results_df['primes'] = results_df['subclassLabel.value'] + ',' + results_df['instanceLabel.value'] + ',' + results_df['partLabel.value'] + ',' + results_df['categoryLabel.value'] + ',' + results_df['propertiesLabel.value'] + ',' + results_df['descLabel.value'] + ',' + results_df['descLabel.noungroups']
  results_df['primes'] = results_df['primes'].apply(lambda x: ','.join(list(set(x.split(',')))))
  #results_df.head(5)
  return results_df

In [9]:
# generate list of universal semantic primes for a given entity by removing empty string values and duplicates
def generate_universal_primes(results_df):
  primes_list = results_df['primes'].values
  primes_string = ','.join(primes_list)
  universal_primes = ','.join(list(set(list(filter(None, primes_string.split(','))))))
  universal_primes = universal_primes.replace('n/a,', '').replace(',n/a', '')
  return universal_primes

In [10]:
# create new dataframe with entity and universal semantic primes
def generate_all_entities_and_primes(entity_and_primes,entity, universal_primes):
  entity_and_primes.append([entity, universal_primes])
  return entity_and_primes

In [11]:
# compute sentence embeddings for semantic universal primes
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [24]:
entity_and_primes = []

for entity in entities:
  entity_id = get_entity_id(entity)
  results_df = get_entity_semantic_information(entity_id)
  results_df = get_description_noun_groups(results_df)
  results_df = generate_semantic_primes(results_df)
  universal_primes = generate_universal_primes(results_df)
  entity_and_primes = generate_all_entities_and_primes(entity_and_primes, entity, universal_primes)

geo_semantic_tower = pd.DataFrame(entity_and_primes, columns=['entity', 'primes'])

geo_semantic_tower["embedding"] = geo_semantic_tower["primes"].apply(get_embedding)

geo_semantic_tower.head(5)

*** ENTITY:  hot spring(s)
*** DICT:  {'id': 'Q655311', 'title': 'Q655311', 'pageid': 616786, 'display': {'label': {'value': 'onsen', 'language': 'en'}, 'description': {'value': 'Japanese hot spring and its associated bathing facilities and inns', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q655311', 'concepturi': 'http://www.wikidata.org/entity/Q655311', 'label': 'onsen', 'description': 'Japanese hot spring and its associated bathing facilities and inns', 'match': {'type': 'alias', 'language': 'en', 'text': 'hot spring'}, 'aliases': ['hot spring']}
*** ENTITY:  flat
*** DICT:  {'id': 'Q188507', 'title': 'Q188507', 'pageid': 187161, 'display': {'label': {'value': 'apartment', 'language': 'en'}, 'description': {'value': 'self-contained housing unit occupying part of a building', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q188507', 'concepturi': 'http://www.wikidata.org/entity/Q188507', 'label': 'apartment', 'descriptio

,entity,primes,embedding
0,hot spring(s),"spring,inns,located in the administrative terr...","[-0.02322094887495041, 0.023217901587486267, -..."
1,flat,"unit,residence,apartment buildings,building pa...","[-0.017850209027528763, -0.00410206476226449, ..."
2,phosphate works,"processing,phosphates, or selling phosphates,i...","[-0.011990244500339031, -0.010169943794608116,..."
3,field(s),"purposes,area of land used for agricultural pu...","[-0.016598518937826157, 0.006747183855623007, ..."
4,icecap dome,"a comparatively elevated area on an icecap,ice...","[-0.00632453290745616, 0.010839623399078846, -..."


In [26]:
# save final dataframe
geo_semantic_tower.to_csv('geo_semantic_primes.csv', index=False)